# RAG Experiments - LLM-GM System Evaluation

This Notebook is used to evaluate the performance of the RAG (Retrieval-Augmented Generation) system.

## Experiment Objectives
1. **Baseline Comparison**: Compare three configurations: No RAG, Random Sampling, and RAG
2. **Ablation Studies**: Test the impact of different parameters (threshold, TOP_K, temperature)
3. **Statistical Analysis**: Calculate significance and confidence intervals
4. **Result Visualization**: Generate plots and tables

## Experiment Configuration
- Test World: RAG Test World
- Runs per configuration: 3runs
- Test scenarios: 10different types of player actions

## 1. Environment Setup

In [ ]:
# Import necessary libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
import time
import os
from datetime import datetime

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully")

In [ ]:
# Import configuration and utility modules
import sys
sys.path.append('..')

from config import (
    SUPABASE_URL, SUPABASE_ANON_KEY, OPENAI_API_KEY,
    TEST_WORLD_NAME, RUNS_PER_CONFIG
)
from utils.rag_simulator import RAGSimulator
from utils.metrics import (
    aggregate_metrics, compare_baselines, perform_t_test,
    calculate_confidence_interval, format_metrics_table,
    calculate_cost_savings
)

# Ensure results directory exists
os.makedirs('results', exist_ok=True)
os.makedirs('results/plots', exist_ok=True)

print("✅ Configuration and utility modules imported successfully")

In [ ]:
# Initialize Supabase and OpenAI clients
from supabase import create_client
from openai import OpenAI

supabase = create_client(SUPABASE_URL, SUPABASE_ANON_KEY)
openai_client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ API clients initialized successfully")
print(f"Supabase URL: {SUPABASE_URL}")
print(f"OpenAI Model: gpt-4.1")

## 2. Load Test Data

In [ ]:
# Load test scenarios
with open('test_scenarios.json', 'r', encoding='utf-8') as f:
    scenarios_data = json.load(f)
    test_scenarios = scenarios_data['scenarios']

print(f"✅ Loaded {len(test_scenarios)} test scenarios")
print("\nTest scenario list:")
for scenario in test_scenarios:
    print(f"  {scenario['id']}. [{scenario['category']}] {scenario['player_message']}")

In [ ]:
# Get test world ID
world_response = supabase.table('worlds')\
    .select('id, name, description')\
    .eq('name', TEST_WORLD_NAME)\
    .single()\
    .execute()

test_world = world_response.data
WORLD_ID = test_world['id']

print(f"✅ Found test world: {test_world['name']}")
print(f"World ID: {WORLD_ID}")
print(f"Description: {test_world['description']}")

In [ ]:
# Check entity counts in the world
entity_types = ['items', 'abilities', 'locations', 'npcs', 'organizations', 'taxonomies', 'rules']
entity_counts = {}

for entity_type in entity_types:
    response = supabase.table(entity_type)\
        .select('id', count='exact')\
        .eq('world_id', WORLD_ID)\
        .execute()
    entity_counts[entity_type] = response.count

print("\nEntity statistics:")
for entity_type, count in entity_counts.items():
    print(f"  {entity_type}: {count}")
print(f"\nTotal entities: {sum(entity_counts.values())}")

In [ ]:
# Helper functions for saving/loading experiment data
def save_experiment_results(data, filename):
    """Save experiment results to JSON file"""
    filepath = f'results/{filename}'
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2, default=str)
    print(f"✅ Experiment results saved to: {filepath}")
    print(f"   Save time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    return filepath

def load_experiment_results(filename):
    """Load experiment results from JSON file"""
    filepath = f'results/{filename}'
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✅ Loaded from backup: {filepath}")
        return data
    else:
        print(f"NO BACKUP FILES: {filepath}")
        return None

print("✅ Helper functions for experiment data save/load defined")


In [ ]:
# Initialize RAG simulator
simulator = RAGSimulator(supabase, openai_client, WORLD_ID)
print("✅ RAG simulator initialized successfully")

## 3. Baseline Comparison Experiment

Compare three configurations:
1. **No RAG**: Pass all world entities
2. **Random Sampling**: Randomly select k entities
3. **RAG**: Semantic similarity retrieval

In [ ]:
# Run baseline comparison experiment
baseline_results = {
    'no_rag': [],
    'random': [],
    'rag': []
}

print("Starting baseline comparison experiment...")
print(f"Runs per configuration {RUNS_PER_CONFIG} runs, in total {len(test_scenarios)} scenarios\n")

total_runs = len(test_scenarios) * RUNS_PER_CONFIG * 3
pbar = tqdm(total=total_runs, desc="Overall progress")

for scenario in test_scenarios:
    player_message = scenario['player_message']
    
    for run in range(RUNS_PER_CONFIG):
        # No RAG
        result = simulator.run_experiment(
            player_message=player_message,
            mode='no_rag'
        )
        result['scenario_id'] = scenario['id']
        result['run'] = run + 1
        baseline_results['no_rag'].append(result)
        pbar.update(1)
        time.sleep(0.5)  # Avoid API rate limiting
        
        # Random Sampling
        result = simulator.run_experiment(
            player_message=player_message,
            mode='random',
            top_k=5
        )
        result['scenario_id'] = scenario['id']
        result['run'] = run + 1
        baseline_results['random'].append(result)
        pbar.update(1)
        time.sleep(0.5)
        
        # RAG
        result = simulator.run_experiment(
            player_message=player_message,
            mode='rag',
            top_k=5,
            similarity_threshold=0.65
        )
        result['scenario_id'] = scenario['id']
        result['run'] = run + 1
        baseline_results['rag'].append(result)
        pbar.update(1)
        time.sleep(0.5)

pbar.close()
print("\n✅ Baseline comparison experiment completed!")

# Save baseline comparison results immediately
save_experiment_results(baseline_results, 'baseline_results_backup.json')
print(f"\nData statistics:")
print(f"   - No RAG: {len(baseline_results['no_rag'])} results")
print(f"   - Random: {len(baseline_results['random'])} results")
print(f"   - RAG: {len(baseline_results['rag'])} results")


In [ ]:
# Check and load baseline comparison results (if not in memory)
if 'baseline_results' not in locals() or not baseline_results.get('no_rag'):
    print("⚠️ baseline_results does not exist or is empty, trying to load from backup...")
    loaded_data = load_experiment_results('baseline_results_backup.json')
    if loaded_data:
        baseline_results = loaded_data
    else:
        print("❌ Backup file does not exist, please run first Cell 11（Baseline Comparison Experiment）")
        raise FileNotFoundError("baseline_results does not exist and no backup file")
else:
    print("✅ baseline_results already exists in memory")

# Check data integrity
print(f"\nData check:")
print(f"  No RAG: {len(baseline_results.get('no_rag', []))} entries")
print(f"  Random: {len(baseline_results.get('random', []))} entries")
print(f"  RAG: {len(baseline_results.get('rag', []))} entries")

if len(baseline_results.get('no_rag', [])) == 0:
    print("⚠️ Warning: baseline_results is empty, please run first Cell 11")


In [ ]:
# Check baseline_results exists, if not try to load from backup
import json
import os

if 'baseline_results' not in locals() or not baseline_results.get('no_rag'):
    print("⚠️ baseline_results does not exist or is empty, trying to load from backup...")
    backup_file = 'results/baseline_results_backup.json'
    if os.path.exists(backup_file):
        with open(backup_file, 'r', encoding='utf-8') as f:
            baseline_results = json.load(f)
        print(f"✅ Loaded from backup: {backup_file}")
    else:
        print("❌ Backup file does not exist, please run first Cell 11（Baseline Comparison Experiment）")
        raise FileNotFoundError("baseline_results does not exist and no backup file")
else:
    print("✅ baseline_results already exists in memory")

# Check data integrity
print(f"\nData check:")
print(f"  No RAG: {len(baseline_results.get('no_rag', []))} entries")
print(f"  Random: {len(baseline_results.get('random', []))} entries")
print(f"  RAG: {len(baseline_results.get('rag', []))} entries")

if len(baseline_results.get('no_rag', [])) == 0:
    print("⚠️ Warning: baseline_results is empty, please run first Cell 11")


In [ ]:
# Analyze baseline comparison results
baseline_comparison = compare_baselines(
    baseline_results['no_rag'],
    baseline_results['random'],
    baseline_results['rag']
)

print("=== Baseline Comparison Results ===")
print(f"\nContext Efficiency: {baseline_comparison.get('context_efficiency_percentage', 0):.2f}% reduction")

print("\n--- No RAG ---")
print(f"Average input tokens: {baseline_comparison['no_rag']['input_tokens']['mean']:.0f}")
print(f"Average latency: {baseline_comparison['no_rag']['latency']['mean']:.2f}s")
print(f"Average cost: ${baseline_comparison['no_rag']['cost']['mean']:.4f}")

print("\n--- Random Sampling ---")
print(f"Average input tokens: {baseline_comparison['random']['input_tokens']['mean']:.0f}")
print(f"Average latency: {baseline_comparison['random']['latency']['mean']:.2f}s")
print(f"Average cost: ${baseline_comparison['random']['cost']['mean']:.4f}")

print("\n--- RAG ---")
print(f"Average input tokens: {baseline_comparison['rag']['input_tokens']['mean']:.0f}")
print(f"Average latency: {baseline_comparison['rag']['latency']['mean']:.2f}s")
print(f"Average cost: ${baseline_comparison['rag']['cost']['mean']:.4f}")

print("\n--- Statistical Significance ---")
for test_name, test_result in baseline_comparison['significance_tests'].items():
    sig = "✅ Significant" if test_result['significant'] else "❌ Not significant"
    print(f"{test_name}: p={test_result['p_value']:.4f} {sig}")

In [ ]:
# Visualize baseline comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

configs = ['no_rag', 'random', 'rag']
config_labels = ['No RAG', 'Random', 'RAG']

# 1. Input tokens comparison
input_tokens = [
    baseline_comparison[c]['input_tokens']['mean']
    for c in configs
]
axes[0, 0].bar(config_labels, input_tokens, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[0, 0].set_ylabel('Average Input Tokens')
axes[0, 0].set_title('Input Tokens Comparison')
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Latency comparison
latencies = [
    baseline_comparison[c]['latency']['mean']
    for c in configs
]
axes[0, 1].bar(config_labels, latencies, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[0, 1].set_ylabel('Average Latency (seconds)')
axes[0, 1].set_title('Latency Comparison')
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Cost comparison
costs = [
    baseline_comparison[c]['cost']['mean']
    for c in configs
]
axes[1, 0].bar(config_labels, costs, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[1, 0].set_ylabel('Average Cost ($)')
axes[1, 0].set_title('Cost Comparison')
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Token distribution box plot
all_input_tokens = [
    [r['input_tokens'] for r in baseline_results[c]]
    for c in configs
]
axes[1, 1].boxplot(all_input_tokens, labels=config_labels)
axes[1, 1].set_ylabel('Input Tokens')
axes[1, 1].set_title('Input Tokens Distribution')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/baseline_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Plot saved to results/plots/baseline_comparison.png")

## 4. Ablation Studies

### 4.1 RAG Threshold Ablation

In [ ]:
# RAG Threshold ablation experiment
threshold_values = [0.5, 0.65, 0.8]


print("Check if there are saved Threshold ablation experiment results...")
loaded_threshold = load_experiment_results('threshold_results_backup.json')
if loaded_threshold:
    # Fix: Convert string keys to float keys
    threshold_results = {float(k): v for k, v in loaded_threshold.items()}
    print(f"✅ Loaded saved results, containing {len(threshold_results)}  threshold's experiment data")
    missing_thresholds = [t for t in threshold_values if not threshold_results.get(t) or len(threshold_results[t]) == 0]
    if missing_thresholds:
        print(f"⚠️ The following threshold data is missing or empty: {missing_thresholds}")
        print("Will continue to run experiments for these thresholds...")
else:
    threshold_results = {t: [] for t in threshold_values}

# Use only the first 5 scenarios to save time
if 'ablation_scenarios' not in locals():
    ablation_scenarios = test_scenarios[:5]

print("\nStarting RAG Threshold ablation experiment...")
print(f"Test thresholds: {threshold_values}\n")

for threshold in threshold_values:
    # Skip if this threshold already has complete data
    if threshold_results.get(threshold) and len(threshold_results[threshold]) > 0:
        expected_count = len(ablation_scenarios) * RUNS_PER_CONFIG
        if len(threshold_results[threshold]) >= expected_count:
            print(f"\nSkip threshold={threshold} (already has {len(threshold_results[threshold])} data entries)")
            continue
    
    print(f"\nTesting threshold={threshold}")
    pbar = tqdm(total=len(ablation_scenarios) * RUNS_PER_CONFIG)
    
    for scenario in ablation_scenarios:
        for run in range(RUNS_PER_CONFIG):
            result = simulator.run_experiment(
                player_message=scenario['player_message'],
                mode='rag',
                similarity_threshold=threshold
            )
            result['scenario_id'] = scenario['id']
            result['run'] = run + 1
            threshold_results[threshold].append(result)
            pbar.update(1)
            time.sleep(0.3)
    
    pbar.close()
    
    # Save immediately after each threshold is completed
    save_experiment_results(threshold_results, 'threshold_results_backup.json')

print("\n✅ RAG Threshold ablation experiment finished!")

In [ ]:
# Analyze Threshold Ablation Result
threshold_comparison = {}
for threshold in threshold_values:
    threshold_comparison[threshold] = aggregate_metrics(threshold_results[threshold])

print("=== RAG Threshold Ablation Result ===")
for threshold in threshold_values:
    metrics = threshold_comparison[threshold]
    print(f"\nThreshold = {threshold}:")
    print(f"  Average retrieved entities: {np.mean([r['total_entities_retrieved'] for r in threshold_results[threshold]]):.1f}")
    print(f"  Average input tokens: {metrics['input_tokens']['mean']:.0f}")
    print(f"  Average latency: {metrics['latency']['mean']:.2f}s")
    print(f"  Average cost: ${metrics['cost']['mean']:.4f}")

In [ ]:
# Visualize Threshold impact
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Retrieved entities
entity_counts = [
    np.mean([r['total_entities_retrieved'] for r in threshold_results[t]])
    for t in threshold_values
]
axes[0].plot(threshold_values, entity_counts, marker='o', linewidth=2, markersize=8)
axes[0].set_xlabel('Similarity Threshold')
axes[0].set_ylabel('Avg Retrieved Entities')
axes[0].set_title('Retrieved Entities vs Threshold')
axes[0].grid(alpha=0.3)

# Input Tokens
input_tokens = [
    threshold_comparison[t]['input_tokens']['mean']
    for t in threshold_values
]
axes[1].plot(threshold_values, input_tokens, marker='s', linewidth=2, markersize=8, color='orange')
axes[1].set_xlabel('Similarity Threshold')
axes[1].set_ylabel('Avg Input Tokens')
axes[1].set_title('Input Tokens vs Threshold')
axes[1].grid(alpha=0.3)

# Costs
costs = [
    threshold_comparison[t]['cost']['mean']
    for t in threshold_values
]
axes[2].plot(threshold_values, costs, marker='^', linewidth=2, markersize=8, color='green')
axes[2].set_xlabel('Similarity Threshold')
axes[2].set_ylabel('Avg Cost ($)')
axes[2].set_title('Cost vs Threshold')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/ablation_threshold.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Plot saved to results/plots/ablation_threshold.png")

### 4.2 TOP_K Ablation

In [ ]:
# TOP_K ablation experiment
top_k_values = [3, 5, 10]


print("Check if there are saved TOP_K ablation experiment result...")
loaded_top_k = load_experiment_results('top_k_results_backup.json')
if loaded_top_k:
    top_k_results = {int(k): v for k, v in loaded_top_k.items()}
    print(f"✅ Loaded saved results, containing {len(top_k_results)} TOP_K value experiment data")
    missing_top_k = [k for k in top_k_values if not top_k_results.get(k) or len(top_k_results[k]) == 0]
    if missing_top_k:
        print(f"⚠️ The following TOP_K values data is missing or empty: {missing_top_k}")
        print("Will continue to run experiments for these TOP_K values...")
else:
    top_k_results = {k: [] for k in top_k_values}

print("\nStarting TOP_K ablation experiment...")
print(f"Test TOP_K values: {top_k_values}\n")

for top_k in top_k_values:
    # Skip if this TOP_K already has complete data
    if top_k_results.get(top_k) and len(top_k_results[top_k]) > 0:
        expected_count = len(ablation_scenarios) * RUNS_PER_CONFIG
        if len(top_k_results[top_k]) >= expected_count:
            print(f"\nSkip TOP_K={top_k} (already has {len(top_k_results[top_k])} data entries)")
            continue
    
    print(f"\nTesting TOP_K={top_k}")
    pbar = tqdm(total=len(ablation_scenarios) * RUNS_PER_CONFIG)
    
    for scenario in ablation_scenarios:
        for run in range(RUNS_PER_CONFIG):
            result = simulator.run_experiment(
                player_message=scenario['player_message'],
                mode='rag',
                top_k=top_k
            )
            result['scenario_id'] = scenario['id']
            result['run'] = run + 1
            top_k_results[top_k].append(result)
            pbar.update(1)
            time.sleep(0.3)
    
    pbar.close()
    
    # Save immediately after each TOP_K is completed
    save_experiment_results(top_k_results, 'top_k_results_backup.json')

print("\n✅ TOP_K ablation experiment Finished!")

In [ ]:
# Analyze TOP_K Ablation Result
top_k_comparison = {}
for k in top_k_values:
    top_k_comparison[k] = aggregate_metrics(top_k_results[k])

print("=== TOP_K ablation results ===")
for k in top_k_values:
    metrics = top_k_comparison[k]
    print(f"\nTOP_K = {k}:")
    print(f"  Average input tokens: {metrics['input_tokens']['mean']:.0f}")
    print(f"  Average latency: {metrics['latency']['mean']:.2f}s")
    print(f"  Average cost: ${metrics['cost']['mean']:.4f}")

### 4.3 Temperature Ablation

In [ ]:
# Temperature ablation experiment
temperature_values = [0.5, 0.8, 1.0]

print("Check if there are saved Temperature ablation experiment results...")
loaded_temperature = load_experiment_results('temperature_results_backup.json')
if loaded_temperature:
    # Fix: Convert string keys to float keys
    temperature_results = {float(k): v for k, v in loaded_temperature.items()}
    print(f"✅ Loaded saved results, containing {len(temperature_results)} 个Temperature value experiment data")
    missing_temp = [t for t in temperature_values if not temperature_results.get(t) or len(temperature_results[t]) == 0]
    if missing_temp:
        print(f"⚠️ The following Temperature values data is missing or empty: {missing_temp}")
        print("Will continue to run experiments for these Temperature values...")
else:
    temperature_results = {t: [] for t in temperature_values}

print("\nStarting Temperature ablation experiment...")
print(f"Test Temperature values: {temperature_values}\n")

for temp in temperature_values:
    # Skip if this Temperature already has complete data
    if temperature_results.get(temp) and len(temperature_results[temp]) > 0:
        expected_count = len(ablation_scenarios) * RUNS_PER_CONFIG
        if len(temperature_results[temp]) >= expected_count:
            print(f"\nSkip Temperature={temp} (already has {len(temperature_results[temp])} data entries)")
            continue
    
    print(f"\nTesting Temperature={temp}")
    pbar = tqdm(total=len(ablation_scenarios) * RUNS_PER_CONFIG)
    
    for scenario in ablation_scenarios:
        for run in range(RUNS_PER_CONFIG):
            result = simulator.run_experiment(
                player_message=scenario['player_message'],
                mode='rag',
                temperature=temp
            )
            result['scenario_id'] = scenario['id']
            result['run'] = run + 1
            temperature_results[temp].append(result)
            pbar.update(1)
            time.sleep(0.3)
    
    pbar.close()
    
    # Save immediately after each Temperature is completed
    save_experiment_results(temperature_results, 'temperature_results_backup.json')

print("\n✅ Temperature ablation experiment finished!")

In [ ]:
# Analyze Temperature Ablation Result
temperature_comparison = {}
for temp in temperature_values:
    temperature_comparison[temp] = aggregate_metrics(temperature_results[temp])

print("=== Temperature Ablation Result ===")
for temp in temperature_values:
    metrics = temperature_comparison[temp]
    print(f"\nTemperature = {temp}:")
    print(f"  Average output tokens: {metrics['output_tokens']['mean']:.0f}")
    print(f"  Output tokens std dev: {metrics['output_tokens']['std']:.0f}")
    print(f"  Average latency: {metrics['latency']['mean']:.2f}s")

## 5. Save Experiment Results

In [ ]:
# Convert to DataFrame and save
import os

os.makedirs('results', exist_ok=True)

# Baseline Comparison Result
baseline_df = pd.DataFrame([
    {
        'config': config,
        'scenario_id': r['scenario_id'],
        'run': r['run'],
        'input_tokens': r['input_tokens'],
        'output_tokens': r['output_tokens'],
        'total_tokens': r['total_tokens'],
        'latency': r['latency'],
        'entities_retrieved': r.get('total_entities_retrieved', 0)
    }
    for config in ['no_rag', 'random', 'rag']
    for r in baseline_results[config]
])
baseline_df.to_csv('results/baseline_comparison.csv', index=False)
print("✅ Baseline comparison results saved to results/baseline_comparison.csv")

# Threshold Ablation Result
threshold_df = pd.DataFrame([
    {
        'threshold': threshold,
        'scenario_id': r['scenario_id'],
        'run': r['run'],
        'input_tokens': r['input_tokens'],
        'entities_retrieved': r['total_entities_retrieved']
    }
    for threshold in threshold_values
    for r in threshold_results[threshold]
])
threshold_df.to_csv('results/ablation_rag_threshold.csv', index=False)
print("✅ Threshold ablation results saved to results/ablation_rag_threshold.csv")

# TOP_K Abaltion Result
top_k_df = pd.DataFrame([
    {
        'top_k': k,
        'scenario_id': r['scenario_id'],
        'run': r['run'],
        'input_tokens': r['input_tokens'],
        'latency': r['latency']
    }
    for k in top_k_values
    for r in top_k_results[k]
])
top_k_df.to_csv('results/ablation_top_k.csv', index=False)
print("✅ TOP_K ablation results saved to results/ablation_top_k.csv")

# Temperature Abaltion Result
temperature_df = pd.DataFrame([
    {
        'temperature': temp,
        'scenario_id': r['scenario_id'],
        'run': r['run'],
        'output_tokens': r['output_tokens'],
        'latency': r['latency']
    }
    for temp in temperature_values
    for r in temperature_results[temp]
])
temperature_df.to_csv('results/ablation_temperature.csv', index=False)
print("✅ Temperature ablation results saved to results/ablation_temperature.csv")

## 6. Generate Statistical Analysis Report

In [ ]:
# Generate Statistical Analysis Report
report_lines = []
report_lines.append("# RAG Experiment Statistical Analysis Report\n")
report_lines.append(f"Experiment Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
report_lines.append(f"Test World: {TEST_WORLD_NAME}\n")
report_lines.append(f"Number of test scenarios: {len(test_scenarios)}\n")
report_lines.append(f"Runs per configuration: {RUNS_PER_CONFIG}\n\n")

report_lines.append("## 1. Baseline Comparison Result\n\n")
report_lines.append(f"Context Efficiency: {baseline_comparison.get('context_efficiency_percentage', 0):.2f}% reduction\n\n")

for config in ['no_rag', 'random', 'rag']:
    metrics = baseline_comparison[config]
    report_lines.append(f"### {config.upper()}\n")
    report_lines.append(f"- Input Tokens: {metrics['input_tokens']['mean']:.2f} ± {metrics['input_tokens']['std']:.2f}\n")
    report_lines.append(f"- Latency: {metrics['latency']['mean']:.3f}s ± {metrics['latency']['std']:.3f}s\n")
    report_lines.append(f"- Cost: ${metrics['cost']['mean']:.5f} ± ${metrics['cost']['std']:.5f}\n\n")

report_lines.append("## 2. Statistic Significant Testing\n\n")
for test_name, test_result in baseline_comparison['significance_tests'].items():
    report_lines.append(f"### {test_name}\n")
    report_lines.append(f"- t-statistic: {test_result['t_statistic']:.4f}\n")
    report_lines.append(f"- p-value: {test_result['p_value']:.4f}\n")
    report_lines.append(f"- Significant (p < 0.05): {test_result['significant']}\n\n")

with open('results/statistical_analysis.txt', 'w', encoding='utf-8') as f:
    f.writelines(report_lines)

print("✅ Statistical analysis report saved to results/statistical_analysis.txt")
print("\n".join(report_lines))

In [ ]:
print("\n🎉 Experiment completed!")
print("\nGenerated files:")
print("  - results/baseline_comparison.csv")
print("  - results/ablation_rag_threshold.csv")
print("  - results/ablation_top_k.csv")
print("  - results/ablation_temperature.csv")
print("  - results/statistical_analysis.txt")
print("  - results/plots/baseline_comparison.png")
print("  - results/plots/ablation_threshold.png")
print("\nThese results can be used for paper writing!")